In [ ]:
import os
from utils import session_config
os.environ['CUDA_VISIBLE_DEVICES'] = "0" # 사용 gpu 선택
session_config.setup_gpus(True, 0.4) # gpu 메모리 사용량 세팅

In [ ]:
%config Completer.use_jedi = False

In [ ]:
from data import coco
from utils.anchor import generate_retina_boxes

In [ ]:
anchor_param = {"ratios": [0.5, 1, 2],
                "scales": [1.0, 1.25,1.58],
                           "fm_sizes": [32, 16, 8, 4, 2, 1],
                           "image_size": 128} #anchor parameters
anchors = generate_retina_boxes(anchor_param)

dataset_root = "/home/files/cnsi-sed-dataset"
dataset = coco.Dataset(dataset_root=dataset_root, 
                       input_shape=(128,182), 
                       label_set=["head","helmet"],
                       anchors=anchors)

annotation_path = "/home/files/cnsi-sed-dataset/files/PIH_yeosu_test/annotations/annotations.json"
train_ds, length = dataset.load_data_generator(annotation_path, is_augment=False, is_box=True, num_examples=-1)

In [ ]:
head = []
helmet = []


def calculate_aspect_ratio(box):
    xmin,ymin,xmax,ymax = box
    ratio = (ymax-ymin)/(xmax-xmin) #너비 대비 높이
    scale = (xmax-xmin)*(ymax-ymin) #이미지 대비 차지넓이
    return scale, ratio
    
    
for filename, image, labels, boxes in train_ds.take(10000):
    for label, box in zip(labels, boxes):
        if label == 1: #head
            scale, ratio = calculate_aspect_ratio(box)
            head.append([scale, ratio])
        elif label == 2: #helmet
            scale, ratio = calculate_aspect_ratio(box)
            helmet.append([scale, ratio])

In [ ]:
from matplotlib import pyplot as plt
import numpy as np



In [ ]:
head = np.array(head)
helmet = np.array(helmet)

In [ ]:
x = head

plt.scatter(x[:,0],
            x[:,1], s=0.5, c='r', label='head')
plt.title("Object scale-ratio scatter graph")
plt.xlabel("Scale : object area by image area")
plt.ylabel("ratio : object height divided by width")
plt.legend()
plt.show()
plt.title("aspect ratio over 1.0")
plt.hist(x[:,1], range=(1,7), bins=10)
plt.ylim(top=20)
plt.show()
plt.title("aspect ratio below 1.0")
plt.hist(x[:,1], range=(0,1), bins=10)
plt.ylim(top=100)
plt.show()

In [ ]:
x = helmet

plt.scatter(x[:,0],
            x[:,1], s=0.5, c='r', label='helmet')
plt.title("Object scale-ratio scatter graph")
plt.xlabel("Scale : object area by image area")
plt.ylabel("ratio : object height divided by width")
plt.legend()
plt.show()
plt.title("aspect ratio over 1.0")
plt.hist(x[:,1], range=(1,7), bins=10)
plt.ylim(top=2000)
plt.show()
plt.title("aspect ratio below 1.0")
plt.hist(x[:,1], range=(0,1), bins=10)
plt.ylim(top=2000)
plt.show()